# Domaći 04 - Parametri kamere

In [1]:
from d04_kamera import *
import numpy as np
import numpy.linalg as LA

## 01

### Kod

In [2]:
def ParametriKamere(T):

    T0 = T[:,0:3]

    if LA.det(T0) < 0:
        T = -T

    Q , R = LA.qr(LA.inv(T0))

    if R[0][0] < 0:
        R[0] = -R[0]
        Q[:,0] = -Q[:,0]
    if R[1][1] < 0:
        R[1] = -R[1]
        Q[:,1] = -Q[:,1]
    if R[2][2] < 0:
        R[2] = -R[2]
        Q[:,2] = -Q[:,2]

    K = LA.inv(R)
    K = K / K[2,2]
    A = np.transpose(Q)

    c1 = LA.det(T[:, 1:])
    c2 = -LA.det(np.array([T[:,0], T[:,2], T[:,3]]).T)
    c3 = LA.det(np.array([T[:,0], T[:,1], T[:,3]]).T)
    c4 = -LA.det(T[:, :-1])

    c1 = c1/c4
    c2 = c2/c4
    c3 = c3/c4

    C = np.array([c1, c2, c3])

    return K , A , C

In [3]:
def Kamera3x4(K, A, C):
    T0 = K @ A
    P = -T0 @ C
    return np.hstack((T0,P.reshape((3,1))))

### Rešenje

In [4]:
n = 208
T = np.array([
    [5, -1 - 2*n, 3, 18 - 3*n],
    [0, -1, 5, 21],
    [0, -1, 0, 1]
])

In [5]:
print("Originalna matrica kamere:\n", T)

K , A , C = ParametriKamere(T)
print("Matrica K:\n", K)
print("Matrica A:\n", A)
print("Koordinate C:\n", C)

T1 = Kamera3x4(K, A, C)
print("Matrica kamere dobijena inverznim postuptkom\n", T1)

Originalna matrica kamere:
 [[   5 -417    3 -606]
 [   0   -1    5   21]
 [   0   -1    0    1]]
Matrica K:
 [[  5.   3. 417.]
 [  0.   5.   1.]
 [  0.   0.   1.]]
Matrica A:
 [[ 1.  0.  0.]
 [ 0.  0.  1.]
 [-0. -1. -0.]]
Koordinate C:
 [207.   1.  -4.]
Matrica kamere dobijena inverznim postuptkom
 [[   5. -417.    3. -606.]
 [   0.   -1.    5.   21.]
 [   0.   -1.    0.    1.]]


## 02

### Kod

In [22]:
def CameraDLP(originali, projekcije):
    
    A = None
    for ((x,y,z,t), (u,v,w)) in zip(originali, projekcije):
        a = np.array([
                [0, 0, 0, 0, -w*x, -w*y, -w*z, -w*t, v*x, v*y, v*z, v*t],
                [w*x, w*y, w*z, w*t, 0, 0, 0, 0, -u*x, -u*y, -u*z, -u*t]
            ])
        
        if A is None:
            A = a
        else:
            A = np.vstack((A, a))

    
    _, _, vt = np.linalg.svd(A)

    T = np.reshape(vt[-1], (3, 4))

    T = T / T[0,0]

    return T

### Rešenje

In [7]:
n = 208

M1 = np.array([460, 280, 250, 1])
M2 = np.array([50, 380, 350, 1])
M3 = np.array([470, 500, 100, 1])
M4 = np.array([380, 630, 50 * n, 1])
M5 = np.array([30 * n, 290, 0, 1])
M6 = np.array([580, 0, 130, 1])

M1p = np.array([288, 251, 1])
M2p = np.array([79, 510, 1])
M3p = np.array([470, 440, 1])
M4p = np.array([520, 590, 1])
M5p = np.array([365, 388, 1])
M6p = np.array([365, 20, 1])

M = [M1, M2, M3, M4, M5, M6]
Mp = [M1p, M2p, M3p, M4p, M5p, M6p]

In [23]:
T2 = CameraDLP(M, Mp)
print(T2)

[[ 1.00000000e+00 -9.49859286e-01  2.10743867e+00 -4.53571847e+02]
 [ 1.09288787e+00 -8.83585872e-02  2.37283385e+00 -9.31602127e+02]
 [ 2.67527975e-03 -2.46399679e-03  4.06504686e-03 -8.82836344e-01]]


## 03

In [9]:
m1 = np.array([167, 52, 235, 1])
m2 = np.array([0, 230, 200, 1])
m3 = np.array([75, 163, 0, 1])
m4 = np.array([273, 112, 80, 1])
m5 = np.array([249, 293, 70, 1])
m6 = np.array([62, 474, 70, 1])

m1p = np.array([922, 535, 1])
m2p = np.array([2349, 607, 1])
m3p = np.array([1856, 1596, 1])
m4p = np.array([952, 1768, 1])
m5p = np.array([2082, 2044, 1])
m6p = np.array([3577, 1667, 1])

m = [m1, m2, m3, m4, m5, m6]
mp = [m1p, m2p, m3p, m4p, m5p, m6p]

In [28]:
T3 = CameraDLP(m, mp)
print("T = \n", T3)

K1 , A1 , C1 = ParametriKamere(T3)
print("K = \n", K1)
print("A = \n", A1)
print("C = \n", C1)

print()
print("Priblizno izmerene koordinate kamere: (565, 450, 380)")

T = 
 [[ 1.00000000e+00 -6.88365639e-01  4.39831001e-01 -3.88712065e+02]
 [-1.14977040e-01 -2.88022565e-02  1.20877983e+00 -3.53431448e+02]
 [ 2.67854042e-04  1.38986736e-04  2.24812167e-04 -2.72238876e-01]]
K = 
 [[ 2.84750481e+03 -4.36543239e+01  1.91421467e+03]
 [ 0.00000000e+00  2.76003584e+03  1.67332045e+03]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
A = 
 [[ 0.44643194 -0.89456764  0.02114848]
 [-0.54224731 -0.25165555  0.80164664]
 [ 0.71180501  0.36934837  0.59742398]]
C = 
 [512.43207838 403.84243122 350.75117265]

Priblizno izmerene koordinate kamere: (565, 450, 380)
